## Analyze coverage_matrix. Either create coverage_matrix from scratch or load from csv

In [29]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))
from python_library import DMD_utilities as DMD_util
from python_library import create_coverage_matrix as create_cm
import pandas as pd

#### Create coverage_matrix_df

In [27]:
reload(DMD_util)
reload(create_cm)
DMD_exons_merged, exon_labels = DMD_util.get_DMD_exons_merged()
subj_name_filter = ['FRMR-00AW-8645', 'FRMR-00DM-853C']
bam_dir = '../../library_files/inputs/bam_files'
matrix_instance = create_cm.coverageMatrix()
coverage_matrix_df = matrix_instance.create_coverage_matrix(DMD_exons_merged, exon_labels, bam_dir=bam_dir, subj_name_filter=subj_name_filter)

The following exons were merged: ['65|66']


Finished parsing all 2 subjects
42215 reads were skipped due to outside_of_exon
773 reads were skipped due to MAPQ below 60


#### Alternatively, load the coverage_matrix from csv

In [30]:
coverage_matrix_df = pd.read_csv("../exon_data/coverage_matrix.csv", index_col=0)

In [31]:
coverage_matrix_df.head()

,id,subject,specimen,sample,gender,sequencer,flow_cell_id,lane,bwa_version,date_modified,...,Ex70,Ex71,Ex72,Ex73,Ex74,Ex75,Ex76,Ex77,Ex78,Ex79
0,FCLR-GP01-2121_1-M1-1_HGGF5AFXX-L001,FCLR-GP01-2121,FCLR-GP01-2121_1,FCLR-GP01-2121_1-M1-1,F,M1,HGGF5AFXX,L001,0.7.13-r1126,1.466796e+09,...,102,44,74,77,139,58,63,77,23,81
1,FCLR-GP01-2121_1-M1-1_HGGF5AFXX-L002,FCLR-GP01-2121,FCLR-GP01-2121_1,FCLR-GP01-2121_1-M1-1,F,M1,HGGF5AFXX,L002,0.7.13-r1126,1.466796e+09,...,85,65,76,57,120,50,95,78,26,67
2,FCLR-GP01-2121_1-M1-1_HGGF5AFXX-L003,FCLR-GP01-2121,FCLR-GP01-2121_1,FCLR-GP01-2121_1-M1-1,F,M1,HGGF5AFXX,L003,0.7.13-r1126,1.466796e+09,...,142,53,75,52,137,51,78,85,22,38
3,FCLR-GP01-2121_1-M1-1_HGGF5AFXX-L004,FCLR-GP01-2121,FCLR-GP01-2121_1,FCLR-GP01-2121_1-M1-1,F,M1,HGGF5AFXX,L004,0.7.13-r1126,1.466796e+09,...,151,34,96,71,117,46,76,88,29,56
4,MESB-0009-690N_ACGT.1.P1.1,MESB-0009-690N,MESB-0009-690N_ACGT_1,MESB-0009-690N_ACGT.1.P1.1,M,P1,NaN,NaN,0.7.13-r1126,1.466796e+09,...,36,22,48,74,64,40,48,10,1,6


#### Add columns for summed coverage across all exons

In [9]:
coverage_matrix_df['all_exons'] = coverage_matrix_df.filter(regex=('Ex')).sum(axis=1)

#### Print summary stats for each exon across all female subjects

In [12]:
female_coverage_matrix = coverage_matrix_df[coverage_matrix_df['gender'] == 'F']
female_coverage_matrix.filter(regex=('Ex|all_exons')).describe().T

,count,mean,std,min,25%,50%,75%,max
Ex1,4.0,429.50,20.469489,408.0,414.75,429.0,443.75,452.0
Ex2,4.0,59.25,14.545904,47.0,50.75,55.0,63.50,80.0
Ex3,4.0,127.50,22.752289,100.0,115.00,128.5,141.00,153.0
Ex4,4.0,187.50,14.387495,173.0,178.25,185.5,194.75,206.0
Ex5,4.0,99.75,15.261608,87.0,88.50,96.0,107.25,120.0
Ex6,4.0,154.75,9.535023,143.0,150.50,155.0,159.25,166.0
Ex7,4.0,270.50,15.022206,257.0,260.75,267.0,276.75,291.0
Ex8,4.0,222.50,12.556539,207.0,215.25,224.0,231.25,235.0
Ex9,4.0,78.25,9.394147,67.0,72.25,80.0,86.00,86.0
Ex10,4.0,149.50,16.663333,134.0,137.75,146.5,158.25,171.0
